
# Assignment 4: Embedding Models, Dense Retrieval, and RAG

**Student names**: _Your_names_here_ <br>
**Group number**: _Your_group_here_ <br>
**Date**: _Submission Date_

## Important notes
Please carefully read the following notes and consider them for the assignment delivery. Submissions that do not fulfill these requirements will not be assessed and should be submitted again.
1. You may work in groups of maximum 2 students.
2. The assignment must be delivered in ipynb format.
3. The assignment must be typed. Handwritten assignments are not accepted.

**Due date**: 26.10.2025 23:59

In this assignment, you will:
- Build a vector search index over a blog corpus using sentence embeddings
- Implement dense retrieval (cosine similarity)
- Use the vector index as the foundation for a simple Retrieval-Augmented Generation (RAG) chat system with evaluation on three queries



---
## Dataset

You will use the blog files, provided in the folder: 
- `blogs-sample` (in the same directory as this notebook)

Use only the blog files provided in the folder below. Each file contains multiple `<post>` elements. Treat **each `<post>` as a separate document**.

**The code to parse files is not provided. Implement the loading yourself in 4.1.**



## 4.1 – Load and parse blog documents

Load all XML files from `blogs-sample`, extract the text of each `<post>`, and store one string per document. Keep the raw text per post as the document text.

You may experience some trouble parsing all lines in the files, but this is okay.



In [ ]:

# TODO: Load and parse the blog posts into a list named `documents`.

# Your code here



## 4.2 – Embedding Models

Select and load a sentence embedding model (e.g., `sentence-transformers/all-MiniLM-L6-v2`) and compute embeddings for all documents.

- Store document embeddings in a variable named `doc_embeddings`.
- Ensure that the same model will be used for query encoding later.

**Report**:
- The embedding matrix shape 


In [ ]:

# TODO: Load a sentence embedding model and encode all documents into `doc_embeddings`.
# You may use `sentence-transformers`. Report the embedding matrix shape.

# Your code here



## 4.3 – Dense Retrieval

Implement a cosine similarity search over `doc_embeddings` for a given query.

- Write a function `dense_search(query: str, k: int = 5) -> list[int]` that returns the indices of the top-k documents.
- Use the same embedding model to encode the query.
- Use cosine similarity for ranking.

**Report**:
- Results for the provided query showing the indices of the top results.


In [ ]:

# TODO: Implement dense retrieval using cosine similarity.
# Function signature to implement:
# def dense_search(query: str, k: int = 5) -> list[int]:

# Your code here

#Report
print(dense_search("How do people feel about their jobs?", k=5))



## 4.4 – Build a Vector Search Index

Build a lightweight vector index structure to enable repeated querying efficiently.

- You may reuse `doc_embeddings` directly or create an index structure. Ensure the index can return top-k document indices given a query vector.


In [ ]:

# TODO: Initialize a vector index over `doc_embeddings`
# Keep code minimal. The goal is to enable fast top-k retrieval for repeated queries.

# Your code here



## 4.5 – RAG (Retrieval-Augmented Generation)

Implement a simple RAG pipeline that:
1) Retrieves the top-k documents for a user query using your vector index.
2) Builds a prompt that includes the query and the retrieved document snippets.
3) Uses a text generation model (your choice) to produce an answer grounded in the retrieved snippets.

- Implement a function `rag_answer(query: str, k: int = 5) -> str`.
- Keep the prompt simple and state clearly that the model should rely on the provided context.


In [ ]:

# TODO: Implement a minimal RAG pipeline.
# Steps (sketch):
# - Use `dense_search` to get top-k indices.

# Your code here


## 4.6 – Evaluation

Use the following queries for your evaluation. For each query:

- Run `dense_search(query, k=5)` to retrieve relevant documents.
- Use `rag_answer(query, k=5)` to generate an answer using the top-5 retrieved documents.

**Queries:**
1. How do people deal with breakups?
2. What do bloggers write about their daily routines?
3. How do people feel about their jobs?


In [ ]:
# Do not change this code
queries = [
    "How do people deal with breakups?",
    "What do bloggers write about their daily routines?",
    "How do people feel about their jobs?"
]

In [ ]:
# TODO: Run and report your evaluation as described above.

def run_batch_evaluation(queries, k=5):
    for i, query in enumerate(queries, 1):
        print("=" * 100)
        print(f"Q{i}: {query}")
        print("-" * 100)

        top_k = dense_search(query, k=k)
        print(f"Top-{k} retrieved indices:", top_k)
        print("\nTop retrieved snippets:")
        for idx in top_k:
            snippet = documents[idx].replace("\n", " ").strip()
            print(f"[{idx}] {snippet[:200]}...\n")

        print("RAG answer:\n")
        answer = rag_answer(query, k=k)
        print(answer)
        print("\n")

# Run the evaluation
run_batch_evaluation(queries, k=5)